Extract Data


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("Credit Card Approval - Extract") \
    .getOrCreate()

file_path = "dbfs:/FileStore/tables/credit_card_approvals.csv"
table_name = "credit_card_approvals"

df = spark.read.csv(file_path, header=True, inferSchema=True)

df.write.format("delta").mode("overwrite").saveAsTable(table_name)

Validate Data

In [0]:
from pyspark.sql.functions import col, when, count
# 1. Checking for Missing Values
missing_values = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

# 2. Checking Data Types
data_types = df.dtypes

numerical_columns = ['Age', 'Debt', 'YearsEmployed', 'CreditScore', 'ZipCode', 'Income']
categorical_columns = ['Gender', 'Married', 'BankCustomer', 'Industry', 'Ethnicity', 'PriorDefault', 'Employed', 'DriversLicense', 'Citizen', 'Approved']

# 3. Range Checks
range_checks = {c: df.filter((col(c) < 0) | col(c).isNull()).count() for c in numerical_columns}

# 4. Categorical Data Consistency
categorical_values = {c: df.select(c).distinct().collect() for c in categorical_columns}

# 5. Checking for Duplicates
duplicates = df.count() - df.dropDuplicates().count()

# Summarizing the validation checks
validation_summary = {
    "Missing Values": missing_values.collect()[0],
    "Data Types": data_types,
    "Range Checks": range_checks,
    "Categorical Values": categorical_values,
    "Duplicates": duplicates
}

validation_summary


{'Missing Values': Row(Gender=0, Age=0, Debt=0, Married=0, BankCustomer=0, Industry=0, Ethnicity=0, YearsEmployed=0, PriorDefault=0, Employed=0, CreditScore=0, DriversLicense=0, Citizen=0, ZipCode=0, Income=0, Approved=0),
 'Data Types': [('Gender', 'int'),
  ('Age', 'double'),
  ('Debt', 'double'),
  ('Married', 'int'),
  ('BankCustomer', 'int'),
  ('Industry', 'string'),
  ('Ethnicity', 'string'),
  ('YearsEmployed', 'double'),
  ('PriorDefault', 'int'),
  ('Employed', 'int'),
  ('CreditScore', 'int'),
  ('DriversLicense', 'int'),
  ('Citizen', 'string'),
  ('ZipCode', 'int'),
  ('Income', 'int'),
  ('Approved', 'int')],
 'Range Checks': {'Age': 0,
  'Debt': 0,
  'YearsEmployed': 0,
  'CreditScore': 0,
  'ZipCode': 0,
  'Income': 0},
 'Categorical Values': {'Gender': [Row(Gender=1), Row(Gender=0)],
  'Married': [Row(Married=1), Row(Married=0)],
  'BankCustomer': [Row(BankCustomer=1), Row(BankCustomer=0)],
  'Industry': [Row(Industry='Education'),
   Row(Industry='Energy'),
   Row(Ind